In [1]:

# Code to read csv file into colaboratory:# Code  
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'10Xf2Xv_pfnIycH2HpNZR3SwWHiHdrqzX'}) # replace the id with id of file you want to access
downloaded.GetContentFile('handwritten.csv')  

#3. Read file as panda dataframe
import pandas as pd
data = pd.read_csv('handwritten.csv')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
!pip install -q keras
!pip install -q numpy
!pip install -q sklearn
!pip install -q matplotlib
!pip install -q pandas

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
%matplotlib inline

from sklearn.model_selection import train_test_split

/Users/Ryan/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load the Data

In [5]:
# Get the data from the csv file
data = data.values
np.random.shuffle(data)

# Split the data into X, Y, and parition the values into the training and test splits
# Reshape X to be in the shape (N, 28, 28)
X, Y = data[:,1:], data[:,0]
# X = X.reshape(X.shape[0], 28, 28)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

print("Train {} {}".format(X_train.shape, Y_train.shape))
print("Test {} {}".format(X_test.shape, Y_test.shape))

Train (297629, 784) (297629,)
Test (74408, 784) (74408,)


# Data Prep

In [6]:
classes = np.unique(Y_train)
n_classes = len(classes)
print('Total number of outputs : ', n_classes)
print('Output classes : ', classes)

Total number of outputs :  26
Output classes :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


In [0]:
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

classes = np.unique(Y_train)
num_classes = len(classes)

X_train = X_train.reshape(-1, img_size, img_size, 1)
X_test = X_test.reshape(-1, img_size, img_size, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

## Normalization

In [0]:
mean_image = np.mean(X_train, axis=0).astype(np.int64)
X_train = (X_train - mean_image)/255
X_test = (X_test - mean_image)/255

In [9]:
from keras.utils import to_categorical

Y_train_onehot = to_categorical(Y_train)
Y_test_onehot = to_categorical(Y_test)
print("Original Label: {}".format(Y_train[0]))
print("One Hot Label: {}".format(Y_train_onehot[0]))

Using TensorFlow backend.


Original Label: 2
One Hot Label: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [10]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train_onehot, test_size=0.2)

print("Train: {} {}".format(X_train.shape, Y_train.shape))
print("Valid: {} {}".format(X_valid.shape, Y_valid.shape))

Train: (238103, 28, 28, 1) (238103, 26)
Valid: (59526, 28, 28, 1) (59526, 26)


# Keras Inits

In [4]:
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.layers import LeakyReLU, ELU
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras import initializers
from tensorflow.python.keras.losses import mean_absolute_error

#Add whatever you need here

# Models

### [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]

In [5]:
def model1():
    model = Sequential()
    '''
    initializer: he_normal
    A more recent paper on this topic, Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification by He et al., 
    derives an initialization specifically for ReLU neurons, reaching the conclusion that the variance of neurons 
    in the network should be 2.0/n. This gives the initialization w = np.random.randn(n) * sqrt(2.0/n), 
    and is the current recommendation for use in practice in the specific case of neural networks with ReLU neurons.
    '''
    #convolutional layer
    model.add(Conv2D(kernel_size=7, strides=1, filters=16, padding='same',
                     activation='linear', name='layer_conv1', input_shape=img_shape_full))
    #reLU activation
    model.add(LeakyReLU(alpha=0.1))
    #Pooling Layer
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))

    model.add(Flatten())
    #Fully-Connected Layer with 128 outputs Adding this in improved scores
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.1))
    #Fully-Connected Layer with Softmax
    model.add(Dense(num_classes, activation='softmax'))

    '''
    For multiclass classification problems like MNIST, cross entropy is typically used as the loss metric
    '''
    model.compile(optimizer=Adam(lr=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
# Insert model here

### [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]

In [0]:
def model2():
  optimizer = Adam(lr=1e-3)

  model = Sequential()

  #conv
  #keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, 
      #dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', 
      #bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, 
      #activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
  model.add(Conv2D(filters=16, kernel_size=8, strides = 1, padding='same', activation='linear',
                   bias_initializer='zeros', input_shape=(img_size,img_size,1)))

  #ReLu
  model.add(LeakyReLU(alpha=0.1))

  #conv2
  model.add(Conv2D(filters=16, kernel_size=8, strides = 2, padding='same', activation='linear',
                   bias_initializer='zeros', input_shape=(img_size,img_size,1)))

  #ReLu
  model.add(LeakyReLU(alpha=0.1))

  #MaxPool
  #keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)
  model.add(MaxPooling2D(strides=2, padding='same'))

  #Affine
  model.add(Flatten())

  #keras.layers.Dense(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', 
      #bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, 
      #activity_regularizer=None, kernel_constraint=None, bias_constraint=None)
  #model.add(Dense(56, activation="relu"))

  model.add(Dense(num_classes, activation='softmax'))

  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

### [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]

In [0]:
def model3():
  pass
# Insert model here

# Training

In [0]:
models = []

In [0]:
mod1 = model1()
mod1.fit(x=X_train,
          y=Y_train,
          epochs=5, batch_size=128,verbose=1, validation_data=(X_valid, Y_valid))
models.append(mod1)

In [21]:
#model2().summary()
mod2 = model2()
mod2.fit(x=X_train,
          y=Y_train,
          epochs=2, batch_size=128,verbose=1, validation_data=(X_valid, Y_valid))
models.append(mod2)

Train on 238103 samples, validate on 59526 samples
Epoch 1/2
238103/238103 [==============================] - 31s 129us/step - loss: 0.2482 - acc: 0.9320 - val_loss: 0.1222 - val_acc: 0.9683
Epoch 2/2
 49920/238103 [=====>........................] - ETA: 22s - loss: 0.1141 - acc: 0.9701238103/238103 [==============================] - 30s 127us/step - loss: 0.1060 - acc: 0.9717 - val_loss: 0.1173 - val_acc: 0.9691


In [0]:
#mod3 = model3()
#mod3.fit(x=X_train,
#           y=Y_train,
#           epochs=5, batch_size=128,verbose=1, validation_data=(X_valid, Y_valid))
#models.append(mod3)

# Ensemble

In [46]:
# print(mod2.predict(X_test))
# mod2Pred = mod2.predict(X_test)
# type(mod2Pred)
print(mod2Pred.shape)
preds = tf.argmax(np.mean([mod.predict(X_test) for mod in models], axis=0),1)
# print(preds.shape)
# print(preds)
# Y_pred = tf.Session().run(preds)
print(Y_pred)
print(Y_test)

incorrect = 0
for i in range(len(Y_pred)):
  if Y_pred[i] != Y_test[i]:
    incorrect += 1
print("Accuracy:", 1.0 * (len(Y_pred)-incorrect) / len(Y_pred) )

#print(mean_absolute_error(Y_test, Y_pred))

(74408, 26)
[14 17 18 ...  3 17 19]
[14 17 18 ...  3 17 19]
Accuracy: 0.9698150736479948
